In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

In [2]:
# 加载训练集
train = pd.read_csv('data/train_dataset_v2.tsv', sep='\t', error_bad_lines=False, warn_bad_lines=False)
print(train.shape)
train.head()

(42790, 4)


,id,content,character,emotions
0,1171_0001_A_1,天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,o2,"0,0,0,0,0,0"
1,1171_0001_A_2,天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,c1,"0,0,0,0,0,0"
2,1171_0001_A_3,o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。,o2,"0,0,0,0,0,0"
3,1171_0001_A_4,o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。,c1,"0,0,0,0,0,0"
4,1171_0001_A_5,o2停下来接过c1手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道...,o2,"0,0,0,0,0,0"


In [3]:
# 加载测试集
test = pd.read_csv('data/test_dataset.tsv', sep='\t')
print(test.shape)
test.head()

(21376, 3)


,id,content,character
0,34170_0002_A_12,穿着背心的b1醒来，看看手机，三点了。,b1
1,34170_0002_A_14,b1走出卧室。,b1
2,34170_0003_A_16,b1拿着手机，点开计时功能。,b1
3,34170_0003_A_17,b1站在淋浴头下面，水从b1的头和脸上冲刷而过。,b1
4,34170_0003_A_18,b1摈着呼吸。,b1


In [4]:
submit = pd.read_csv('data/submit_example.tsv', sep='\t')
print(submit.shape)
submit.head()

(21376, 2)


,id,emotion
0,34170_0002_A_12,"0,0,0,0,0,0"
1,34170_0002_A_14,"0,0,0,0,0,0"
2,34170_0003_A_16,"0,0,0,0,0,0"
3,34170_0003_A_17,"0,0,0,0,0,0"
4,34170_0003_A_18,"0,0,0,0,0,0"


In [5]:
train = train[train['emotions'].notna()]
print(train.shape)

(36782, 4)


In [6]:
def get_context(df, cur_id):
    pos = cur_id.split('_')[-1]
    # print(pos)
    pos_str = str(int(pos) - 1)

    location = train['id'][0][:-2] + '_' + pos_str
    #print(location)
    ret = train.loc[train['id'] == location]['content'].values.tolist()
    if len(ret) == 1:
        return ret[0]
    else:
        return "无上文"
print(get_context(train, '1171_0001_A_1'))

无上文


In [7]:
train['character'].fillna('无角色', inplace=True)
test['character'].fillna('无角色', inplace=True)

train['text'] = train['content'].astype(str) + ' 角色: ' + train['character'].astype(str)
test['text'] = test['content'].astype(str) + ' 角色: ' + test['character'].astype(str)

In [8]:
# add train context info
for i in range(len(train['text'])):
    try:
        lcw = get_context(train, train['id'][i])
    except:
        continue
    train['text'][i] += ' 上文: ' + lcw
train.head()

,id,content,character,emotions,text
0,1171_0001_A_1,天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,o2,"0,0,0,0,0,0",天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。 角色: ...
1,1171_0001_A_2,天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,c1,"0,0,0,0,0,0",天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。 角色: ...
2,1171_0001_A_3,o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。,o2,"0,0,0,0,0,0",o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。 角色: o2 上文: 天空下着暴雨...
3,1171_0001_A_4,o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。,c1,"0,0,0,0,0,0",o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。 角色: c1 上文: o2一手拿着...
4,1171_0001_A_5,o2停下来接过c1手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道...,o2,"0,0,0,0,0,0",o2停下来接过c1手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道...


In [9]:
# add test context info
for i in range(len(test['id'])):
    try:
        lcw = get_context(test, test['id'][i])
    except:
        continue
    test['text'][i] += ' 上文: ' + lcw
test.head()

,id,content,character,text
0,34170_0002_A_12,穿着背心的b1醒来，看看手机，三点了。,b1,穿着背心的b1醒来，看看手机，三点了。 角色: b1 上文: o2笑了笑：军礼不是这么敬的。...
1,34170_0002_A_14,b1走出卧室。,b1,b1走出卧室。 角色: b1 上文: o2示范了一个动作，c1照做。
2,34170_0003_A_16,b1拿着手机，点开计时功能。,b1,b1拿着手机，点开计时功能。 角色: b1 上文: c1照做。
3,34170_0003_A_17,b1站在淋浴头下面，水从b1的头和脸上冲刷而过。,b1,b1站在淋浴头下面，水从b1的头和脸上冲刷而过。 角色: b1 上文: b1画外音：我叫b1...
4,34170_0003_A_18,b1摈着呼吸。,b1,b1摈着呼吸。 角色: b1 上文: 无上文


In [10]:
train['labels'] = train['emotions'].apply(lambda x: [int(i) for i in x.split(',')])

In [11]:
train_data = train[['text', 'labels']].copy()
train_data.head()

,text,labels
0,天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。 角色: ...,"[0, 0, 0, 0, 0, 0]"
1,天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。 角色: ...,"[0, 0, 0, 0, 0, 0]"
2,o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。 角色: o2 上文: 天空下着暴雨...,"[0, 0, 0, 0, 0, 0]"
3,o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。 角色: c1 上文: o2一手拿着...,"[0, 0, 0, 0, 0, 0]"
4,o2停下来接过c1手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道...,"[0, 0, 0, 0, 0, 0]"


In [12]:
#split train : dev = 9 : 1
train_data_raw = train_data.sample(frac=1.0, random_state=42)
train_data = train_data_raw[:33100]
dev_data = train_data_raw[33100:]
print(len(train_data), len(dev_data))

33100 3682


In [13]:
print(len(train_data), len(dev_data))

33100 3682


In [14]:
from simpletransformers_clf.classification import (
    MultiLabelClassificationModel as MultiLabelClassificationModelClf,
    MultiLabelClassificationArgs
)
from simpletransformers_reg.classification import (
    MultiLabelClassificationModel as MultiLabelClassificationModelReg,
    MultiLabelClassificationArgs as MultiLabelClassificationArgsReg
)

In [15]:
model_args = MultiLabelClassificationArgs()
model_args.max_seq_length = 128
model_args.num_train_epochs = 3
model_args.learning_rate = 3e-5
model_args.do_lower_case = True
model_args.overwrite_output_dir = True
model_args.no_save = True
model_args.save_model_every_epoch = False
model_args.save_steps = -1

In [ ]:
#model = MultiLabelClassificationModel('bert', 'hfl/chinese-bert-wwm-ext', num_labels=6)
model = MultiLabelClassificationModelClf('bert', 'hfl/chinese-roberta-wwm-ext', num_labels=6, args=model_args)
model.train_model(train_data)
# eval

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForMultiLabelSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification

  0%|          | 0/33100 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/4138 [00:00<?, ?it/s]

In [28]:
predictions, raw_outputs = model.predict([text for text in dev_data['text'].values])
# raw_outputs[raw_outputs<=0.01] = 0.0
# raw_outputs[raw_outputs >= 3.01] = 3.0

from sklearn.metrics import mean_squared_error
result = raw_outputs.tolist()
test_y = dev_data['labels'].values.tolist()
rmse = np.sqrt(mean_squared_error(result, test_y))
score = 1.0 / (1+rmse)
print(f"得分：{score}")

  0%|          | 0/3310 [00:00<?, ?it/s]

  0%|          | 0/414 [00:00<?, ?it/s]

得分：0.7313271537885129


In [29]:
import torch
torch.save(model, f'roberta_best_{score}_clf.pt')
torch.save(model2, f'roberta_best_{score2}_reg.pt')

In [25]:
model_args2 = MultiLabelClassificationArgsReg()
model_args2.max_seq_length = 128
model_args2.num_train_epochs = 3
model_args2.learning_rate = 3e-5
model_args2.do_lower_case = True
model_args2.overwrite_output_dir = True
model_args2.no_save = True
model_args2.save_model_every_epoch = False
model_args2.save_steps = -1
#model = MultiLabelClassificationModel('bert', 'hfl/chinese-bert-wwm-ext', num_labels=6)
model2 = MultiLabelClassificationModelReg('bert', 'hfl/chinese-roberta-wwm-ext', num_labels=6, args=model_args2)
model2.train_model(train_data)
# eval

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForMultiLabelSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification

  0%|          | 0/33104 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/4138 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/4138 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/4138 [00:00<?, ?it/s]

(12414, 0.17814469151005682)

In [26]:
predictions, raw_outputs2 = model2.predict([text for text in dev_data['text'].values])
raw_outputs[raw_outputs<=0.001] = 0.0
raw_outputs[raw_outputs >= 3.001] = 3.0

from sklearn.metrics import mean_squared_error
result2 = raw_outputs2.tolist()
test_y = dev_data['labels'].values.tolist()
rmse2 = np.sqrt(mean_squared_error(result2, test_y))
score2 = 1.0 / (1+rmse2)
print(f"得分：{score2}")

  0%|          | 0/3310 [00:00<?, ?it/s]

  0%|          | 0/414 [00:00<?, ?it/s]

得分：0.7661568740532065


In [30]:
result3 = result2.copy()
for i in range(len(result3)):
    for j in range(len(result3[i])):
        result3[i][j] = result[i][j] * result2[i][j]
rmse3 = np.sqrt(mean_squared_error(result3, test_y))
score3 = 1.0 / (1+rmse3)
print(f"得分：{score3}")

得分：0.7624120964566801


In [34]:
# test
_, raw_outputs1 = model.predict([text for text in test['text'].values])
_, raw_outputs2 = model2.predict([text for text in test['text'].values])

prob_submit = submit.copy()
to_submit = []
to_submit2 = []
to_submit3 = []
for i in range(len(raw_outputs)):
    lst = raw_outputs1[i].tolist()
    to_submit.append(','.join([str(i) for i in lst]))
    lst2 = raw_outputs2[i].tolist()
    to_submit2.append(','.join([str(i) for i in lst2]))
    lst3 = len(lst) * [0]
    for j in range(len(lst)):
        lst3[j] = lst[j] * lst2[j]
    to_submit3.append(','.join([str(i) for i in lst3]))
    # break
prob_submit['emotion'] = to_submit #prob_submit['emotion'].apply(lambda x: ','.join([str(i) for i in x]))
prob_submit.head()
prob_submit.to_csv(f'0927_s7313_roberta_clf_structured.tsv', sep='\t', index=False)
prob_submit['emotion'] = to_submit2 #prob_submit['emotion'].apply(lambda x: ','.join([str(i) for i in x]))
prob_submit.head()
prob_submit.to_csv(f'0927_s7661_roberta_reg_structured.tsv', sep='\t', index=False)
prob_submit['emotion'] = to_submit3
prob_submit.head()
prob_submit.to_csv(f'0927_s7624_roberta_mix_structured.tsv', sep='\t', index=False)

  0%|          | 0/21376 [00:00<?, ?it/s]

  0%|          | 0/2672 [00:00<?, ?it/s]

  0%|          | 0/21376 [00:00<?, ?it/s]

  0%|          | 0/2672 [00:00<?, ?it/s]

ValueError: Length of values (3310) does not match length of index (21376)

In [ ]:
train_data.values[3]
# # sub probs
# prob_submit = sub.copy()
# to_submit = []
# # prob_submit['emotion'] = raw_outputs
# for i in range(len(raw_outputs)):
#     lst = raw_outputs[i].tolist()
#     to_submit.append(','.join([str(i * 3) for i in lst]))
#     # break
# prob_submit['emotion'] = to_submit #prob_submit['emotion'].apply(lambda x: ','.join([str(i) for i in x]))
# prob_submit.head()
# prob_submit.to_csv('4epoch_2e05_probs_emplified_roberta.tsv', sep='\t', index=False)

In [ ]:
# model = MultiLabelClassificationModel('bert', 'outputs', num_labels=6)

In [ ]:
# from simpletransformers.classification import ClassificationModel, ClassificationArgs
# import pandas as pd
# import logging


# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)
# train_data_0 = []
# train_data_1 = []
# train_data_2 = []
# train_data_3 = []
# train_data_4 = []
# train_data_5 = []

# for i in range(len(train_data['text'])):
#     try:
#         ## train data
#         train_data_0.append([train_data['text'][i], train_data['labels'][i][0]])
#         train_data_1.append([train_data['text'][i], train_data['labels'][i][1]])
#         train_data_2.append([train_data['text'][i], train_data['labels'][i][2]])
#         train_data_3.append([train_data['text'][i], train_data['labels'][i][3]])
#         train_data_4.append([train_data['text'][i], train_data['labels'][i][4]])
#         train_data_5.append([train_data['text'][i], train_data['labels'][i][5]])
#     except:
#         continue

# dev_data_0 = []
# dev_data_1 = []
# dev_data_2 = []
# dev_data_3 = []
# dev_data_4 = []
# dev_data_5 = []

# for i in range(len(dev_data['text'])):
#     try:
#         ## train data
#         dev_data_0.append([dev_data['text'][i], dev_data['labels'][i][0]])
#         dev_data_1.append([dev_data['text'][i], dev_data['labels'][i][1]])
#         dev_data_2.append([dev_data['text'][i], dev_data['labels'][i][2]])
#         dev_data_3.append([dev_data['text'][i], dev_data['labels'][i][3]])
#         dev_data_4.append([dev_data['text'][i], dev_data['labels'][i][4]])
#         dev_data_5.append([dev_data['text'][i], dev_data['labels'][i][5]])
#     except:
#         continue

In [ ]:
# train_df_0 = pd.DataFrame(train_data_0)
# train_df_0.columns = ["text", "labels"]
# train_df_1 = pd.DataFrame(train_data_1)
# train_df_1.columns = ["text", "labels"]
# train_df_2 = pd.DataFrame(train_data_2)
# train_df_2.columns = ["text", "labels"]
# train_df_3 = pd.DataFrame(train_data_3)
# train_df_3.columns = ["text", "labels"]
# train_df_4 = pd.DataFrame(train_data_4)
# train_df_4.columns = ["text", "labels"]
# train_df_5 = pd.DataFrame(train_data_5)
# train_df_5.columns = ["text", "labels"]

In [ ]:
# dev_df_0 = pd.DataFrame(dev_data_0)
# dev_df_0.columns = ["text", "labels"]
# dev_df_1 = pd.DataFrame(dev_data_1)
# dev_df_1.columns = ["text", "labels"]
# dev_df_2 = pd.DataFrame(dev_data_2)
# dev_df_2.columns = ["text", "labels"]
# dev_df_3 = pd.DataFrame(dev_data_3)
# dev_df_3.columns = ["text", "labels"]
# dev_df_4 = pd.DataFrame(dev_data_4)
# dev_df_4.columns = ["text", "labels"]
# dev_df_5 = pd.DataFrame(dev_data_5)
# dev_df_5.columns = ["text", "labels"]


In [ ]:
# # Optional model configuration
# model_args_0 = ClassificationArgs(learning_rate = 2e-5, num_train_epochs=4, output_dir='outputs_0')

# # Create a ClassificationModel
# model_0 = ClassificationModel(
#     'bert',
#     'hfl/chinese-roberta-wwm-ext',
#     num_labels=4,
#     args=model_args_0
# ) 
# # Train the model
# model_0.train_model(train_df_0)

In [ ]:
# # Optional model configuration
# model_args_1 = ClassificationArgs(learning_rate = 2e-5, num_train_epochs=4, output_dir='outputs_1')

# # Create a ClassificationModel
# model_1 = ClassificationModel(
#     'bert',
#     'hfl/chinese-roberta-wwm-ext',
#     num_labels=4,
#     args=model_args_1
# ) 
# # Train the model
# model_1.train_model(train_df_1)

In [ ]:
# # Optional model configuration
# model_args_2 = ClassificationArgs(learning_rate = 2e-5, num_train_epochs=4, output_dir='outputs_2')

# # Create a ClassificationModel
# model_2 = ClassificationModel(
#     'bert',
#     'hfl/chinese-roberta-wwm-ext',
#     num_labels=4,
#     args=model_args_2
# ) 
# # Train the model
# model_2.train_model(train_df_2)

In [ ]:
# # Optional model configuration
# model_args_3 = ClassificationArgs(learning_rate = 2e-5, num_train_epochs=4, output_dir='outputs_3')

# # Create a ClassificationModel
# model_3 = ClassificationModel(
#     'bert',
#     'hfl/chinese-roberta-wwm-ext',
#     num_labels=4,
#     args=model_args_3
# ) 
# # Train the model
# model_3.train_model(train_df_3)

In [ ]:
# # Optional model configuration
# model_args_4 = ClassificationArgs(learning_rate = 2e-5, num_train_epochs=4, output_dir='outputs_4')

# # Create a ClassificationModel
# model_4 = ClassificationModel(
#     'bert',
#     'hfl/chinese-roberta-wwm-ext',
#     num_labels=4,
#     args=model_args_4
# ) 
# # Train the model
# model_4.train_model(train_df_4)

In [ ]:
# # Optional model configuration
# model_args_5 = ClassificationArgs(learning_rate = 2e-5, num_train_epochs=4, output_dir='outputs_5')

# # Create a ClassificationModel
# model_5 = ClassificationModel(
#     'bert',
#     'hfl/chinese-roberta-wwm-ext',
#     num_labels=4,
#     args=model_args_5
# ) 
# # Train the model
# model_5.train_model(train_df_5)

In [ ]:
# model_0 = ClassificationModel('bert', 'outputs_0', num_labels=4)
# model_1 = ClassificationModel('bert', 'outputs_1', num_labels=4)
# model_2 = ClassificationModel('bert', 'outputs_2', num_labels=4)
# model_3 = ClassificationModel('bert', 'outputs_3', num_labels=4)
# model_4 = ClassificationModel('bert', 'outputs_4', num_labels=4)
# model_5 = ClassificationModel('bert', 'outputs_5', num_labels=4)

In [ ]:
# # multi-classification
# _, raw_output_0 = model_0.predict([text for text in dev_data['text'].values])
# _, raw_output_1 = model_1.predict([text for text in dev_data['text'].values])
# _, raw_output_2 = model_2.predict([text for text in dev_data['text'].values])
# _, raw_output_3 = model_3.predict([text for text in dev_data['text'].values])
# _, raw_output_4 = model_4.predict([text for text in dev_data['text'].values])
# _, raw_output_5 = model_5.predict([text for text in dev_data['text'].values])


In [ ]:
# import numpy as np
# import math

# # custom function
# def sigmoid(x):
#     return 1 / (1 + math.exp(-x))

# # define vectorized sigmoid
# sigmoid_v = np.vectorize(sigmoid)

# raw_output_0 = sigmoid_v(raw_output_0).tolist()
# raw_output_1 = sigmoid_v(raw_output_1).tolist()
# raw_output_2 = sigmoid_v(raw_output_2).tolist()
# raw_output_3 = sigmoid_v(raw_output_3).tolist()
# raw_output_4 = sigmoid_v(raw_output_4).tolist()
# raw_output_5 = sigmoid_v(raw_output_5).tolist()


In [ ]:

# # eval on joint model
# predictions, raw_outputs = model.predict([text for text in dev_data['text'].values])
# raw_outputs[raw_outputs<=0.01] = 0

# from sklearn.metrics import mean_squared_error
# result = raw_outputs.tolist()
# for i in range(len(result)):
#     for j in range(6):
#         if result[i][j] > 0.95:
#             if j == 0:
#                 max_prob = max(raw_output_0[i])
#                 max_indx = raw_output_0[i].index(max_prob)
#                 if max_indx == 0:
#                     continue
#                 result[i][j] = max_prob * max_indx
#             elif j == 1:
#                 max_prob = max(raw_output_1[i])
#                 max_indx = raw_output_1[i].index(max_prob)
#                 if max_indx == 0:
#                     continue
#                 result[i][j] = max_prob * max_indx
#             elif j == 2:
#                 max_prob = max(raw_output_2[i])
#                 max_indx = raw_output_2[i].index(max_prob)
#                 if max_indx == 0:
#                     continue
#                 result[i][j] = max_prob * max_indx
#             elif j == 3:
#                 max_prob = max(raw_output_3[i])
#                 max_indx = raw_output_3[i].index(max_prob)
#                 if max_indx == 0:
#                     continue
#                 result[i][j] = max_prob * max_indx
#             elif j == 4:
#                 max_prob = max(raw_output_4[i])
#                 max_indx = raw_output_4[i].index(max_prob)
#                 if max_indx == 0:
#                     continue
#                 result[i][j] = max_prob * max_indx
#             elif j == 5:
#                 max_prob = max(raw_output_5[i])
#                 max_indx = raw_output_5[i].index(max_prob)
#                 if max_indx == 0:
#                     continue
#                 result[i][j] = max_prob * max_indx
# test_y = dev_data['labels'].values.tolist()
# rmse = np.sqrt(mean_squared_error(result, test_y))
# score = 1.0 / (1+rmse)
# print(f"得分：{score}")

In [ ]:
# # multi-classification
# _, raw_output_0 = model_0.predict([text for text in test['text'].values])
# _, raw_output_1 = model_1.predict([text for text in test['text'].values])
# _, raw_output_2 = model_2.predict([text for text in test['text'].values])
# _, raw_output_3 = model_3.predict([text for text in test['text'].values])
# _, raw_output_4 = model_4.predict([text for text in test['text'].values])
# _, raw_output_5 = model_5.predict([text for text in test['text'].values])

In [ ]:
# raw_output_0 = sigmoid_v(raw_output_0).tolist()
# raw_output_1 = sigmoid_v(raw_output_1).tolist()
# raw_output_2 = sigmoid_v(raw_output_2).tolist()
# raw_output_3 = sigmoid_v(raw_output_3).tolist()
# raw_output_4 = sigmoid_v(raw_output_4).tolist()
# raw_output_5 = sigmoid_v(raw_output_5).tolist()

In [ ]:

# # test
# predictions, raw_outputs = model.predict([text for text in test['text'].values])
# raw_outputs[raw_outputs<=0.01] = 0
# # sub probs
# prob_submit = submit.copy()
# to_submit = []
# result = raw_outputs.tolist()
# for i in range(len(result)):
#     for j in range(6):
#         if result[i][j] > 0.95:
#             if j == 0:
#                 max_prob = max(raw_output_0[i])
#                 max_indx = raw_output_0[i].index(max_prob)
#                 if max_indx == 0:
#                     continue
#                 result[i][j] = max_prob * max_indx
#             elif j == 1:
#                 max_prob = max(raw_output_1[i])
#                 max_indx = raw_output_1[i].index(max_prob)
#                 if max_indx == 0:
#                     continue
#                 result[i][j] = max_prob * max_indx
#             elif j == 2:
#                 max_prob = max(raw_output_2[i])
#                 max_indx = raw_output_2[i].index(max_prob)
#                 if max_indx == 0:
#                     continue
#                 result[i][j] = max_prob * max_indx
#             elif j == 3:
#                 max_prob = max(raw_output_3[i])
#                 max_indx = raw_output_3[i].index(max_prob)
#                 if max_indx == 0:
#                     continue
#                 result[i][j] = max_prob * max_indx
#             elif j == 4:
#                 max_prob = max(raw_output_4[i])
#                 max_indx = raw_output_4[i].index(max_prob)
#                 if max_indx == 0:
#                     continue
#                 result[i][j] = max_prob * max_indx
#             elif j == 5:
#                 max_prob = max(raw_output_5[i])
#                 max_indx = raw_output_5[i].index(max_prob)
#                 if max_indx == 0:
#                     continue
#                 result[i][j] = max_prob * max_indx
# # prob_submit['emotion'] = raw_outputs
# for i in range(len(result)):
#     lst = result[i]
#     to_submit.append(','.join([str(i) for i in lst]))
#     # break
# prob_submit['emotion'] = to_submit #prob_submit['emotion'].apply(lambda x: ','.join([str(i) for i in x]))
# prob_submit.head()
# prob_submit.to_csv('0922_s6905_bert_fusion.tsv', sep='\t', index=False)

In [ ]:
print('a')

In [9]:
print(1)

1
